In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import wecorules as weco
from datetime import datetime

In [7]:
data = pd.read_csv("C:\\Users\\Sampreeth Salveru\\Downloads\\AG_Discrete_DS06-Jul-2020.csv")
excel = pd.read_excel("C:\\Users\\Sampreeth Salveru\\Downloads\\riskQA.xlsx",index_col="Quality Attributes")
data['Date'] = pd.to_datetime(data['PSDate'])
data['Date'] = data.Date.dt.strftime('%Y-%m-%d')
filter_col = [col for col in data if col.startswith(('Drug','Load'))]

In [3]:
for i in range(len(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30))):
    x = np.array(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30)[i])
    search = np.array(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30)[i].index)[0]
    search1 = np.array(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30)[i].index)[-1]
    beg = data.Date[search]
    end = data.Date[search1]
    mn = np.mean(x)
    std = np.std(x)
    leng = len(x)
#     print(mn,std)
#     print(leng, mn, std)
    print(beg+" | "+end)

2015-04-10 | 2015-10-21
2015-10-24 | 2017-03-24
2017-03-26 | 2018-05-02
2018-05-05 | 2019-04-13
2019-04-16 | 2019-12-21


In [26]:

def cpkppk(col):
    mean = excel.loc[col,"Mean"]
    ucl = excel.loc[col,"UCL"]
    lcl = excel.loc[col,"LCL"]
    usl = excel.loc[col,"USL"]
    lsl = excel.loc[col,"LSL"]
    units = excel.loc[col,"Units"]
    count1 = len(weco.weco1(col, ucl, lcl))
    count4 = len(weco.weco4(col, mean))
    count5 = len(weco.weco5(col))
    total_count = count1 + count4 + count5
    if total_count > 0:
        percent1 = count1/total_count
    work = np.array([])
    work1 = np.array([])

    for i in range(len(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30))):
        x = np.array(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30)[i])
        search = np.array(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30)[i].index)[0]
        search1 = np.array(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30)[i].index)[-1]
        beg = data.Date[search]
        end = data.Date[search1]    
        mn = np.mean(x)
        std = np.std(x)
        
        if total_count == 0:
            if np.isnan(usl) == False and np.isnan(lsl) == False:
                cpkcalc = [(usl - mn)/(3*std), (mn-lsl)/(3*std)]
                cpk = min(cpkcalc)
            elif np.isnan(usl) == True:
                cpk = (mn-lsl)/(3*std)
            elif np.isnan(lsl) == True:
                cpk = (usl-mn)/(3*std)  
        elif percent1 >= 0.02:
            if np.isnan(usl) == False and np.isnan(lsl) == False:
                ppkcalc = [(usl - mn)/(3*std), (mn-lsl)/(3*std)]
                ppk = min(ppkcalc)
                work = np.append(work, ppk)
                work1 = np.append(work1, "ppk")
            elif np.isnan(usl) == True:
                ppk = (mn - lsl)/(3*std)
            elif np.isnan(lsl) == True:
                ppk = (usl - mn)/(3*std)
        elif percent1 <= 0.02:
            if np.isnan(usl) == False and np.isnan(lsl) == False:
                cpkcalc = [(usl - mn)/(3*std), (mn-lsl)/(3*std)]
                cpk = min(cpkcalc)
            elif np.isnan(usl) == True:
                cpk = (mn-lsl)/(3*std)
            elif np.isnan(lsl) == True:
                cpk = (usl-mn)/(3*std)
    
    return work,work1
        

cpkppk(col = 'DrugSubstance_Sodium')



(array([0.63048439, 0.86933562, 0.78167982, 0.79611734, 0.97669317]),
 array(['ppk', 'ppk', 'ppk', 'ppk', 'ppk'], dtype='<U32'))

In [14]:
len(weco.weco4('DrugSubstance_Sodium',mean=27.8))

47

In [27]:
for x in filter_col[0:5]:
    cpkppk(x)

C:\conda\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars


In [16]:
df

,Param,calc,cpk/ppk
